# Adding metadata

Most cases come with a README file, which is a good thing. However, it is not a structured and standardized metadata resource. Better and good practice is to provide a JSON-LD file. We can use `pivmetadlib` to write it.

Consider Case A from the PIV Challenge 2001. It has the following information, which was combined (scattered metadata should be avoided) on the website and the README.txt file:

We can identify the following parts in those information:
- personal information (contact)
- case information (title, date, abstract)
- camera information (characteristics)
- PIV parameters used for the reference analysis
- description on how data should be provided (this is

In [ ]:
from piv2hdf import pivmetalib
from h5rdmtoolbox.convention import metadata

## Personal information (contact information)

In [ ]:
contact = pivmetalib.Person(mbox="christian.kaehler@dlr.de")

## Case information

In [ ]:
caseinfo = pivmetalib.DescriptiveData(title="Loss of seeding in the core of a tip vortex",
                                         description="""Image A001_1.tif and A001_2.tif were recorded at the DNW-LLF in order
to study experimentally the wake vortex formation behind a transport
aircraft (DLR ALVAST half model) in landing configuration (U=60 m/s,
main flow direction is perpendicular to the light-sheet plane). The
measurement position was 1.64 m behind the wing tip and the field of
view is 170 mm by 140 mm. The images were selected as strong gradients,
loss of image density, and varying particle image sizes are common
problems for many PIV applications in large wind tunnels.""",
                                         keywords=["piv", "piv-challenge"],
                                         dates={'uploaded': '26.10.2000'}) # https://schema.org/uploadDate

## Camera Info

In [ ]:
# camera = pivmetalib.Camera(type

In [ ]:
pivmetalib.HorizontalSensorResolutio

In [ ]:
pivmetalib.HorizontalSensorResolution(value=1024, units='px')

In [ ]:
pco = pivmetalib.Organization(name='pco', url="http://www.pco.de")

In [ ]:
# quantum_efficiency = pivmetalib.Variable(
#     label = 'quantum efficiency', # =prefLabel =HDF dataset name
#     value = 0.1,  # HDF5 data
#     standard_name ='https://matthiasprobst.github.io/piv-convention#quantum_efficiency',
#     # ...
# )

In [ ]:
# quantum_efficiency = pivmetalib.Variable(
#     label = 'quantum efficiency', # =prefLabel =HDF dataset name
#     value = 0.1,  # HDF5 data
#     standard_name = {'name': 'quantum_efficiency',
#                      'standard_name_table': 'https://url/to/my/snt',
#                      'dbpedia':"https://dbpedia.org/page/Quantum_efficiency",
#                      'quantity_kind':None}
#     # ...
# )

In [ ]:
# quantum_efficiency = pivmetalib.Variable(
#     label = 'quantum efficiency', # =prefLabel =HDF dataset name
#     value = 0.1,  # HDF5 data
#     standard_name = pivmetalib.utils.sn_from_snt('quantum_efficiency', table='url')
#     # ...
# )

In [ ]:
qe = pivmetalib.QuantumEfficiency(value=0.1)

In [ ]:
# camera can be real or synthetic, should there be a difference?
# A camera is a tool. all parameters are Values which will be interprted as "hasParameter"
# if a parameter is defined, then it has a standard name
camera = pivmetalib.Camera(type="PCO SensiCam",
                            manufacturer=pco,
                            horizontal_sensor_resolution=pivmetalib.HorizontalSensorResolution(value=1024, units='px'),
                            vertical_sensor_resolution=pivmetalib.VerticalSensorResolution(value=1024, units='px'),
                            # vertical_sensor_resolution=1024,
                            sensor_pixel_width={'value': 6.7, 'units': 'um'},
                            sensor_pixel_height={'value': 6.7, 'units': 'um'},
                            # dynamic_range=12, # bits
                            quantum_efficiency=qe,
                            # quantum_efficiency={'value': 0.1,
                            #                    'standard_name': 'https://matthiasprobst.github.io/piv-convention#quantum_efficiency'}, # s.n. is defined as an individual in ontology. otherwise could by
                            # # free entries:
                            full_well_capacity='25000 e',
                            readout_noise='@12.5 MHz: 7...8 e'
                            # horizontal_ccd_dill_ratio=None, # for synthetic
                            # vertical_ccd_dill_ratio=None, # for synthetic
                           )
# print(camera.jsonld_dump())

In [ ]:
print(camera.jsonld_dump(indent=2))

In [ ]:
from piv2hdf.pivmetalib.utils import is_m4itool

# import abc
# class Individual(abc.ABC):
#     @abc.abstractmethod
#     def json_dump():
#         pass
    

# def parse_individual():
#     pass

def jsonld_dump_many(*models):
    """

    TODO: parse such that hasParameter is set

    We have Tools and Variables as generic types together with the generic concept of the StandardName
    we can give sense to the tool and method parameters! That's the idea behind it!

    Also StandardName uses quantityKind instead of canoncial unit! More state of the art!

    Digital camera is not needed. It is a tool with standardized Parameters (=Variable with standard name)

    """
    context = {}
    for model in models:
        context.update(model._context)
    d = {'@context': context,
        '@graph': []}
    
    for i, model in enumerate(models):

        d['@graph'].append(json.loads(model.jsonld_dump(exclude_context=True)))
        
        # modeltype = model._type
        # data = model.get_data()  
        # extra_data = model._extra_fields
        
        # # print(modeltype, is_m4itool(modeltype, context))
        # if is_m4itool(modeltype, context):
        #     # every field is a m4i:Variable or m4i:Organization
        #     for field in model._model.model_fields:
        #         print(field)
        #     # for field in data.
        
        # if modeltype is None:
        #     d['@graph'].append({'@id': f'_:b{i}', **data, **extra_data})
        # else:
        #     d['@graph'].append({'@id': f'_:b{i}', '@type': modeltype, **data, **extra_data})
    # pprint(d)
    return d

import json

with open('pivmetadata.jsonld', 'w') as f:
    json.dump(jsonld_dump_many(contact, camera, caseinfo), f, indent=2)

In [ ]:
# camera._model.model_fields['manufacturer']

In [ ]:
import h5rdmtoolbox as h5tbx

with h5tbx.File() as h5:
    data = h5.create_group('data')
    metadata = h5.create_group('metadata')
    
    case_info = metadata.create_group('case_info')
    caseinfo.write(case_info)
    
    
    contacts = metadata.create_group('contacts/contact')
    contact.write(contacts)

    cam_grp = metadata.create_group('instruments/camera')
    camera.write(cam_grp)
    
    h5.dump(True)
filename = h5.hdf_filename

In [ ]:
from h5rdmtoolbox.wrapper.jsonld import dumps

## Dump the content of the HDF5 file into a JSON-LD file

You may reformat it with [JSON-LD Playground](https://json-ld.org/playground/)

In [ ]:
with h5tbx.File(filename) as h5:
    print(dumps(h5, indent=2))